In [1]:
import os
import csv
import pandas as pd
import numpy as np

In [2]:
kaggle_file = os.pardir + '/output_data/merged_kaggle_data.csv'
sb_file = os.pardir + '/output_data/stolen_base_data.csv'
retrosheet_players_file = os.pardir + '/input_data/biofile.txt'

output_file = os.pardir + '/output_data/merged_sb_kaggle_data.csv'

In [3]:
kaggle_df = pd.read_csv(kaggle_file)
sb_df = pd.read_csv(sb_file, keep_default_na=False)
retrosheet_players_df = pd.read_csv(retrosheet_players_file)

In [4]:
# restrict to players who (possibly) played in 2018
retrosheet_players_df['PLAY DEBUT'] = pd.to_datetime(retrosheet_players_df['PLAY DEBUT'])
retrosheet_players_df['PLAY LASTGAME'] = pd.to_datetime(retrosheet_players_df['PLAY LASTGAME'])
retrosheet_players_df = retrosheet_players_df[(retrosheet_players_df['PLAY DEBUT'] <= pd.to_datetime('2019-01-01')) & 
                      (retrosheet_players_df['PLAY LASTGAME'] >= pd.to_datetime('2018-01-01'))]
retrosheet_players_df = retrosheet_players_df.reset_index()

In [5]:
# use retrosheet player ids to add player names to stolen base data
retrosheet_players_df['name'] = retrosheet_players_df['NICKNAME'] + ' ' + retrosheet_players_df['LAST']
retrosheet_players_df = retrosheet_players_df[['PLAYERID', 'name', 'BATS', 'THROWS', 'HEIGHT', 'WEIGHT']]

In [6]:
retrosheet_players_df[retrosheet_players_df.duplicated(subset=['name'], keep=False)]

,PLAYERID,name,BATS,THROWS,HEIGHT,WEIGHT
3,adama001,Austin Adams,R,R,5-11,190.0
4,adama002,Austin Adams,R,R,6-02,225.0
547,guerj002,Javy Guerra,R,R,6-00,205.0
549,guerj004,Javy Guerra,L,R,5-11,155.0
1066,ramij003,Jose Ramirez,B,R,5-09,165.0
1067,ramij004,Jose Ramirez,R,R,6-03,190.0


In [7]:
retrosheet_pitcher_names = [None] * len(sb_df)
retrosheet_pitcher_throws = [None] * len(sb_df)
retrosheet_pitcher_heights = [None] * len(sb_df)
retrosheet_pitcher_weights = [None] * len(sb_df)

retrosheet_catcher_names = [None] * len(sb_df)
retrosheet_catcher_throws = [None] * len(sb_df)
retrosheet_catcher_heights = [None] * len(sb_df)
retrosheet_catcher_weights = [None] * len(sb_df)

retrosheet_batter_names = [None] * len(sb_df)
retrosheet_batter_bats = [None] * len(sb_df)
retrosheet_batter_heights = [None] * len(sb_df)
retrosheet_batter_weights = [None] * len(sb_df)

retrosheet_runner_on_first_names = [None] * len(sb_df)
retrosheet_runner_on_first_heights = [None] * len(sb_df)
retrosheet_runner_on_first_weights = [None] * len(sb_df)

retrosheet_runner_on_second_names = [None] * len(sb_df)
retrosheet_runner_on_second_heights = [None] * len(sb_df)
retrosheet_runner_on_second_weights = [None] * len(sb_df)

retrosheet_runner_on_third_names = [None] * len(sb_df)
retrosheet_runner_on_third_heights = [None] * len(sb_df)
retrosheet_runner_on_third_weights = [None] * len(sb_df)

In [8]:
for i in range(len(sb_df)):
    
    pitcher_index = retrosheet_players_df[retrosheet_players_df['PLAYERID'] == sb_df['pitcher'][i]].index[0]
    retrosheet_pitcher_names[i] = retrosheet_players_df.iloc[pitcher_index]['name']
    retrosheet_pitcher_throws[i] = retrosheet_players_df.iloc[pitcher_index]['THROWS']
    retrosheet_pitcher_heights[i] = retrosheet_players_df.iloc[pitcher_index]['HEIGHT']
    retrosheet_pitcher_weights[i] = retrosheet_players_df.iloc[pitcher_index]['WEIGHT']
    
    catcher_index = retrosheet_players_df[retrosheet_players_df['PLAYERID'] == sb_df['catcher'][i]].index[0]
    retrosheet_catcher_names[i] = retrosheet_players_df.iloc[catcher_index]['name']
    retrosheet_catcher_throws[i] = retrosheet_players_df.iloc[catcher_index]['THROWS']
    retrosheet_catcher_heights[i] = retrosheet_players_df.iloc[catcher_index]['HEIGHT']
    retrosheet_catcher_weights[i] = retrosheet_players_df.iloc[catcher_index]['WEIGHT']
    
    batter_index = retrosheet_players_df[retrosheet_players_df['PLAYERID'] == sb_df['batter'][i]].index[0]
    retrosheet_batter_names[i] = retrosheet_players_df.iloc[batter_index]['name']
    retrosheet_batter_bats[i] = retrosheet_players_df.iloc[batter_index]['BATS']
    retrosheet_batter_heights[i] = retrosheet_players_df.iloc[batter_index]['HEIGHT']
    retrosheet_batter_weights[i] = retrosheet_players_df.iloc[batter_index]['WEIGHT']
    
    if(sb_df['runner_on_first'][i] == 'N/A'):
        retrosheet_runner_on_first_names[i] = 'N/A'
        retrosheet_runner_on_first_heights[i] = 'N/A'
        retrosheet_runner_on_first_weights[i] = 'N/A'
    else:
        runner_on_first_index = retrosheet_players_df[retrosheet_players_df['PLAYERID'] == sb_df['runner_on_first'][i]].index[0]
        retrosheet_runner_on_first_names[i] = retrosheet_players_df.iloc[runner_on_first_index]['name']
        retrosheet_runner_on_first_heights[i] = retrosheet_players_df.iloc[runner_on_first_index]['HEIGHT']
        retrosheet_runner_on_first_weights[i] = retrosheet_players_df.iloc[runner_on_first_index]['WEIGHT']
        
    if(sb_df['runner_on_second'][i] == 'N/A'):
        retrosheet_runner_on_second_names[i] = 'N/A'
        retrosheet_runner_on_second_heights[i] = 'N/A'
        retrosheet_runner_on_second_weights[i] = 'N/A'
    else:
        runner_on_second_index = retrosheet_players_df[retrosheet_players_df['PLAYERID'] == sb_df['runner_on_second'][i]].index[0]
        retrosheet_runner_on_second_names[i] = retrosheet_players_df.iloc[runner_on_second_index]['name']
        retrosheet_runner_on_second_heights[i] = retrosheet_players_df.iloc[runner_on_second_index]['HEIGHT']
        retrosheet_runner_on_second_weights[i] = retrosheet_players_df.iloc[runner_on_second_index]['WEIGHT']
        
    if(sb_df['runner_on_third'][i] == 'N/A'):
        retrosheet_runner_on_third_names[i] = 'N/A'
        retrosheet_runner_on_third_heights[i] = 'N/A'
        retrosheet_runner_on_third_weights[i] = 'N/A'
    else:
        runner_on_third_index = retrosheet_players_df[retrosheet_players_df['PLAYERID'] == sb_df['runner_on_third'][i]].index[0]
        retrosheet_runner_on_third_names[i] = retrosheet_players_df.iloc[runner_on_third_index]['name']
        retrosheet_runner_on_third_heights[i] = retrosheet_players_df.iloc[runner_on_third_index]['HEIGHT']
        retrosheet_runner_on_third_weights[i] = retrosheet_players_df.iloc[runner_on_third_index]['WEIGHT']

In [9]:
sb_df['pitcher_name'] = retrosheet_pitcher_names
sb_df['pitcher_throws'] = retrosheet_pitcher_throws
sb_df['pitcher_height'] = retrosheet_pitcher_heights
sb_df['pitcher_weight'] = retrosheet_pitcher_weights

sb_df['catcher_name'] = retrosheet_catcher_names
sb_df['catcher_throws'] = retrosheet_catcher_throws
sb_df['catcher_height'] = retrosheet_catcher_heights
sb_df['catcher_weight'] = retrosheet_catcher_weights

sb_df['batter_name'] = retrosheet_batter_names
sb_df['batter_bats'] = retrosheet_batter_bats
sb_df['batter_height'] = retrosheet_batter_heights
sb_df['batter_weight'] = retrosheet_batter_weights

sb_df['runner_on_first_name'] = retrosheet_runner_on_first_names
sb_df['runner_on_first_height'] = retrosheet_runner_on_first_heights
sb_df['runner_on_first_weight'] = retrosheet_runner_on_first_weights

sb_df['runner_on_second_name'] = retrosheet_runner_on_second_names
sb_df['runner_on_second_height'] = retrosheet_runner_on_second_heights
sb_df['runner_on_second_weight'] = retrosheet_runner_on_second_weights

sb_df['runner_on_third_name'] = retrosheet_runner_on_third_names
sb_df['runner_on_third_height'] = retrosheet_runner_on_third_heights
sb_df['runner_on_third_weight'] = retrosheet_runner_on_third_weights

In [10]:
print(sb_df.columns)
print(kaggle_df.columns)

Index(['game_id', 'home_team', 'away_team', 'date_time', 'play',
       'is_stolen_base_attempt', 'is_successful', 'inning', 'home_half',
       'outs', 'pitcher', 'catcher', 'batter', 'on_first', 'on_second',
       'on_third', 'runner_on_first', 'runner_on_second', 'runner_on_third',
       'pitches', 'num_pitches', 'balls', 'strikes', 'pitch_num_on_event',
       'strike_on_event', 'swing_on_event', 'pitchout_on_event',
       'blocked_on_event', 'pickoffs_to_first', 'pickoffs_to_second',
       'pickoffs_to_third', 'pitchouts', 'pitches_run_on', 'total_outs',
       'pitcher_name', 'pitcher_throws', 'pitcher_height', 'pitcher_weight',
       'catcher_name', 'catcher_throws', 'catcher_height', 'catcher_weight',
       'batter_name', 'batter_bats', 'batter_height', 'batter_weight',
       'runner_on_first_name', 'runner_on_first_height',
       'runner_on_first_weight', 'runner_on_second_name',
       'runner_on_second_height', 'runner_on_second_weight',
       'runner_on_third_name'

In [11]:
# change 1/0 variables in the kaggle data to True/False variables
kaggle_df = kaggle_df.astype({'on_1b':bool, 'on_2b':bool, 'on_3b':bool})

In [12]:
# change 'top' in the kaggle data to be the same indicator as 'home_half' in the retrosheet data
kaggle_df['top'] = ~kaggle_df['top']
kaggle_df.rename(columns={'top':'home_half'}, inplace=True)

In [13]:
# rename columns to match between the two data sets
sb_df.rename(columns={'balls':'b_count', 'strikes':'s_count'}, inplace=True)

kaggle_df.rename(columns={'pitch_num':'pitch_num_on_event', 'on_1b':'on_first', 
                          'on_2b':'on_second', 'on_3b':'on_third'}, inplace=True)

In [14]:
# identify the pitcher names that don't match between the retrosheet and kaggle data

In [15]:
sb_unique_pitcher_names = pd.DataFrame(sb_df['pitcher_name'].unique(), columns=['pitcher_name'])
sb_unique_pitcher_names.to_csv(os.pardir + '/output_data/player_names/merging_sb_kaggle/sb_pitcher_names.txt', sep='\n', index=False)

In [16]:
kaggle_unique_pitcher_names = pd.DataFrame(kaggle_df['pitcher_name'].unique(), columns=['pitcher_name'])
kaggle_unique_pitcher_names.to_csv(os.pardir + '/output_data/player_names/merging_sb_kaggle/kaggle_pitcher_names.txt', sep='\n', index=False)

In [17]:
# find the pitchers in the retrosheet (sb_df) data that aren't in the kaggle data
missed_pitchers = []
for i in range(len(sb_unique_pitcher_names)):
    current_pitcher = sb_unique_pitcher_names['pitcher_name'][i]
    pitcher_found = False
    for j in range(len(kaggle_unique_pitcher_names)):
        if(kaggle_unique_pitcher_names['pitcher_name'][j] == current_pitcher):
            pitcher_found = True
    if(not pitcher_found):
        missed_pitchers.append(current_pitcher)
print(missed_pitchers)

['J.C. Ramirez', 'Michael Fiers', 'Jorge de la Rosa', 'J.T. Chargois', 'Daniel Winkler', 'Mike Wright', 'Zack Britton', 'Jake Junis', 'Matt Boyd', 'Seung Hwan Oh', 'Daniel Coulombe', 'Jacob Faria', 'Michael Dunn', 'Mark Leiter', 'Tom Milone', 'Nestor Cortes', 'A.J. Ramos', 'Vincent Velasquez', 'Jose Valdez', 'Jose Fernandez']


In [18]:
# RETROSHEET - KAGGLE
# J.C. Ramirez - JC Ramirez
# Michael Fiers - Mike Fiers
# Jorge de la Rosa - Jorge De La Rosa
# J.T. Chargois - JT Chargois
# Daniel Winkler - Dan Winkler
# Mike Wright - Mike Wright Jr. (?)
# Zack Britton - Zach Britton
# Jake Junis - Jakob Junis
# Matt Boyd - Matthew Boyd
# Seung Hwan Oh - Seunghwan Oh
# Daniel Coulombe - Danny Coulombe
# Jacob Faria - Jake Faria
# Michael Dunn - Mike Dunn
# Mark Leiter - Mark Leiter Jr.
# Tom Milone - Tommy Milone
# Nestor Cortes - 'Nestor Cortes ' (extra space)
# A.J. Ramos - AJ Ramos
# Vincent Velasquez - Vince Velasquez
# Jose Valdez - Jose A. Valdez
# Jose Fernandez - Jose Manuel Fernandez

In [19]:
# edit the mismatched names in the kaggle data so they match the retrosheet data
old_pitcher_names = ['JC Ramirez', 'Mike Fiers', 'Jorge De La Rosa', 'JT Chargois', 'Dan Winkler', 'Mike Wright Jr.', 
                      'Zach Britton', 'Jakob Junis', 'Matthew Boyd', 'Seunghwan Oh', 'Danny Coulombe', 'Jake Faria', 
                      'Mike Dunn', 'Mark Leiter Jr.', 'Tommy Milone', 'Nestor Cortes ', 'AJ Ramos', 'Vince Velasquez', 
                      'Jose A. Valdez', 'Jose Manuel Fernandez']
new_pitcher_names = ['J.C. Ramirez', 'Michael Fiers', 'Jorge de la Rosa', 'J.T. Chargois', 'Daniel Winkler', 'Mike Wright', 
                     'Zack Britton', 'Jake Junis', 'Matt Boyd', 'Seung Hwan Oh', 'Daniel Coulombe', 'Jacob Faria', 
                     'Michael Dunn', 'Mark Leiter', 'Tom Milone', 'Nestor Cortes', 'A.J. Ramos', 'Vincent Velasquez', 
                     'Jose Valdez', 'Jose Fernandez']
kaggle_df['pitcher_name'] = kaggle_df['pitcher_name'].replace(old_pitcher_names, new_pitcher_names)

In [20]:
# identify the batter names that don't match between the retrosheet and kaggle data

In [21]:
sb_unique_batter_names = pd.DataFrame(sb_df['batter_name'].unique(), columns=['batter_name'])
sb_unique_batter_names.to_csv(os.pardir + '/output_data/player_names/merging_sb_kaggle/sb_batter_names.txt', sep='\n', index=False)

In [22]:
kaggle_unique_batter_names = pd.DataFrame(kaggle_df['batter_name'].unique(), columns=['batter_name'])
kaggle_unique_batter_names.to_csv(os.pardir + '/output_data/player_names/merging_sb_kaggle/kaggle_batter_names.txt', sep='\n', index=False)

In [23]:
missed_batters = []
for i in range(len(sb_unique_batter_names)):
    current_batter = sb_unique_batter_names['batter_name'][i]
    batter_found = False
    for j in range(len(kaggle_unique_batter_names)):
        if(kaggle_unique_batter_names['batter_name'][j] == current_batter):
            batter_found = True
    if(not batter_found):
        missed_batters.append(current_batter)
print(missed_batters)

['Eric Young', 'Steven Souza', 'J.T. Riddle', 'Nick Castellanos', 'Michael Taylor', 'Yulieski Gurriel', 'Rafael Lopez', 'Matt den Dekker', 'Giovanny Urshela', 'Lourdes Gurriel']


In [24]:
# RETROSHEET - KAGGLE
# Eric Young - Eric Young Jr.
# Steven Souza - Steven Souza Jr.
# J.T. Riddle - JT Riddle
# Nick Castellanos - Nicholas Castellanos
# Michael Taylor - Michael A. Taylor
# Yulieski Gurriel - Yuli Gurriel
# Rafael Lopez - Raffy Lopez
# Matt den Dekker - Matthew den Dekker
# Giovanny Urshela - Gio Urshela
# Lourdes Gurriel - Lourdes Gurriel Jr.

In [25]:
# edit the mismatched names in the kaggle data so they match the retrosheet data
old_batter_names = ['Eric Young Jr.', 'Steven Souza Jr.', 'JT Riddle', 'Nicholas Castellanos', 'Michael A. Taylor', 
                     'Yuli Gurriel', 'Raffy Lopez', 'Matthew den Dekker', 'Gio Urshela', 'Lourdes Gurriel Jr.']
new_batter_names = ['Eric Young', 'Steven Souza', 'J.T. Riddle', 'Nick Castellanos', 'Michael Taylor', 'Yulieski Gurriel', 
                    'Rafael Lopez', 'Matt den Dekker', 'Giovanny Urshela', 'Lourdes Gurriel']
kaggle_df['batter_name'] = kaggle_df['batter_name'].replace(old_batter_names, new_batter_names)

In [26]:
# for the game 'CIN201806230,' fix the pitcher name in the kaggle data
kaggle_df.loc[(kaggle_df['ab_id']==2018086225) & (kaggle_df['pitch_num_on_event']!=4), 'pitcher_name'] = 'Rob Zastryzny'

In [27]:
# for the game 'CIN201807020,' change the start time to '2018-07-02 19:11:00' in the kaggle data
kaggle_df.loc[kaggle_df['g_id']==201801254, 'date_time'] = '2018-07-02 19:11:00'

In [28]:
# merge the data
merged_df = sb_df.merge(kaggle_df, on=['home_team', 'away_team', 'date_time', 'inning', 'home_half', 'outs', 
                                       'pitcher_name', 'batter_name', 'on_first', 'on_second', 'on_third', 
                                       'b_count', 's_count', 'pitch_num_on_event'], how='left', indicator=True)

,game_id,home_team,away_team,date_time,play,is_stolen_base_attempt,is_successful,inning,home_half,outs,...,pfx_x,pfx_z,nasty,zone,code,type,pitch_type,event_num,b_score,_merge
0,ANA201804020,ANA,CLE,2018-04-02 19:07:00,SB2,True,True,3,False,1,...,-3.411778,7.770916,59.0,14.0,B,B,FF,141.0,1.0,both
1,ANA201804040,ANA,CLE,2018-04-04 13:08:00,SB2,True,True,2,False,2,...,6.516427,4.206289,66.0,14.0,C,S,CH,117.0,1.0,both
2,ANA201804040,ANA,CLE,2018-04-04 13:08:00,SB2,True,True,5,False,2,...,0.671294,9.559485,24.0,11.0,B,B,FF,278.0,1.0,both
3,ANA201804060,ANA,OAK,2018-04-06 19:07:00,POCS2(1361),True,False,4,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,ANA201804060,ANA,OAK,2018-04-06 19:07:00,SB2,True,True,8,True,0,...,-11.608520,8.169124,35.0,8.0,B,B,FT,661.0,12.0,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3341,WAS201809230,WAS,NYN,2018-09-23 14:00:00,SB2,True,True,3,True,0,...,10.093280,9.268572,54.0,11.0,B,B,SI,230.0,1.0,both
3342,WAS201809240,WAS,MIA,2018-09-24 19:06:00,SB2.1-3(E2/TH),True,True,4,False,2,...,-11.850087,0.906987,50.0,13.0,S,S,CH,291.0,1.0,both
3343,WAS201809240,WAS,MIA,2018-09-24 19:06:00,SB2,True,True,8,True,2,...,-6.665560,10.061261,36.0,14.0,B,B,FF,621.0,7.0,both
3344,WAS201809250,WAS,MIA,2018-09-25 19:05:00,SB2,True,True,1,True,1,...,6.802498,-1.602704,70.0,7.0,C,S,CU,41.0,0.0,both


In [29]:
left_only_df = merged_df[merged_df['_merge'] == 'left_only']
print(len(left_only_df))

262


In [30]:
# important: events that don't occur on a pitch (e.g., POCS) won't show up in the kaggle data
# these events are indicated with 'pitch_num_on_event = -1'

In [31]:
left_only_df[left_only_df['pitch_num_on_event'] == -1]

,game_id,home_team,away_team,date_time,play,is_stolen_base_attempt,is_successful,inning,home_half,outs,...,pfx_x,pfx_z,nasty,zone,code,type,pitch_type,event_num,b_score,_merge
3,ANA201804060,ANA,OAK,2018-04-06 19:07:00,POCS2(1361),True,False,4,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
28,ANA201806020,ANA,TEX,2018-06-02 18:27:00,SB2,True,True,6,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
35,ANA201806240,ANA,TOR,2018-06-24 13:08:00,POCS3(1456),True,False,8,True,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
80,ANA201809160,ANA,SEA,2018-09-16 13:08:00,POCS2(13),True,False,1,False,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
81,ANA201809160,ANA,SEA,2018-09-16 13:08:00,POCS2(136),True,False,2,True,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3256,WAS201805230,WAS,SDN,2018-05-23 16:05:00,POCS2(136),True,False,8,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3280,WAS201807020,WAS,BOS,2018-07-02 19:08:00,POCS2(134),True,False,8,False,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3306,WAS201808041,WAS,CIN,2018-08-04 13:06:00,POCS2(136),True,False,1,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3309,WAS201808080,WAS,ATL,2018-08-08 19:09:00,SB2,True,True,3,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [32]:
left_only_df[left_only_df['pitch_num_on_event'] != -1]

,game_id,home_team,away_team,date_time,play,is_stolen_base_attempt,is_successful,inning,home_half,outs,...,pfx_x,pfx_z,nasty,zone,code,type,pitch_type,event_num,b_score,_merge


In [33]:
print(merged_df.columns)

Index(['game_id', 'home_team', 'away_team', 'date_time', 'play',
       'is_stolen_base_attempt', 'is_successful', 'inning', 'home_half',
       'outs', 'pitcher', 'catcher', 'batter', 'on_first', 'on_second',
       'on_third', 'runner_on_first', 'runner_on_second', 'runner_on_third',
       'pitches', 'num_pitches', 'b_count', 's_count', 'pitch_num_on_event',
       'strike_on_event', 'swing_on_event', 'pitchout_on_event',
       'blocked_on_event', 'pickoffs_to_first', 'pickoffs_to_second',
       'pickoffs_to_third', 'pitchouts', 'pitches_run_on', 'total_outs',
       'pitcher_name', 'pitcher_throws', 'pitcher_height', 'pitcher_weight',
       'catcher_name', 'catcher_throws', 'catcher_height', 'catcher_weight',
       'batter_name', 'batter_bats', 'batter_height', 'batter_weight',
       'runner_on_first_name', 'runner_on_first_height',
       'runner_on_first_weight', 'runner_on_second_name',
       'runner_on_second_height', 'runner_on_second_weight',
       'runner_on_third_nam

In [34]:
columns_to_drop = ['px', 'pz', 'ax', 'ay', 'az', 'sz_bot', 'sz_top', 'type_confidence', 'vx0', 'vy0', 'vz0', 'x', 'x0', 'y', 'y0', 'z0', 'pfx_x', 'pfx_z', 'code', 'type', 'event_num']
merged_df = merged_df.drop(columns=columns_to_drop)

In [36]:
merged_df.to_csv(output_file, index=False)